# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [18]:
import graphlab
import numpy as np

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [46]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1']=feature

    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name]= poly_sframe['power_1'].apply(lambda x: x**power)
    return poly_sframe
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
sales = graphlab.SFrame('~/Documents/Arjun/Python/Coursera/Course2/Week 2/kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [47]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [93]:
poly_data = polynomial_sframe(sales['sqft_living'], 15)
my_features = poly_data.column_names() # get the name of the features
l2_small_penalty = 1e-5
poly_data['price'] = sales['price'] # add price to the data since it's the target
model = graphlab.linear_regression.create(poly_data, target = 'price', features = my_features,verbose=False,l2_penalty=l2_small_penalty)
model.coefficients.print_rows(num_rows=17)


+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    163969.60002    |   414561.763901   |
|   power_1   |  None |   105.923991078    |   1907.10822222   |
|   power_2   |  None |    0.1443036965    |   3.66121109095   |
|   power_3   |  None | -0.000143575423418 |  0.00386936368751 |
|   power_4   |  None | 5.97621592945e-08  | 2.50251469654e-06 |
|   power_5   |  None | -9.73555598079e-12 | 1.03823080283e-09 |
|   power_6   |  None | 3.68530549834e-16  | 2.79994459456e-13 |
|   power_7   |  None | 4.67483039368e-20  | 4.87159758566e-17 |
|   power_8   |  None | -1.56836118265e-24 | 5.98086694592e-21 |
|   power_9   |  None | -2.69164475514e-28 | 8.66065041226e-25 |
|   power_10  |  None | -3.20193827093e-33 |  1.1856123342e-28 |
|   power_11  |  None | 1.14253667666e-36  |  9.3912751384e-33 |
|   power_12  |  None | 7

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [51]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [52]:
poly1_data = polynomial_sframe(set_1['sqft_living'], 15)
my_features = poly1_data.column_names() # get the name of the features
l2_small_penalty = 1e-5
poly1_data['price'] = set_1['price'] # add price to the data since it's the target
model1 = graphlab.linear_regression.create(poly1_data, target = 'price', verbose=False,features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model1.coefficients.print_rows(num_rows=16)



+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    9306.4606221    |   1255976.06157   |
|   power_1   |  None |   585.865823394    |   6003.28876462   |
|   power_2   |  None |  -0.397305895643   |   11.9613259026   |
|   power_3   |  None | 0.000141470900599  |  0.0131345206929  |
|   power_4   |  None | -1.52945989958e-08 | 8.84846297281e-06 |
|   power_5   |  None | -3.79756325772e-13 | 3.83736176297e-09 |
|   power_6   |  None | 5.97481763253e-17  | 1.08407665704e-12 |
|   power_7   |  None | 1.06888504767e-20  | 1.93500977266e-16 |
|   power_8   |  None | 1.59344027887e-25  | 1.89384579435e-20 |
|   power_9   |  None | -6.92834984105e-29 |        nan        |
|   power_10  |  None | -6.83813287347e-33 | 3.49595202136e-29 |
|   power_11  |  None | -1.62686187633e-37 | 2.50639941958e-33 |
|   power_12  |  None | 2

In [53]:
poly2_data = polynomial_sframe(set_2['sqft_living'], 15)
my_features = poly2_data.column_names() # get the name of the features
l2_small_penalty = 1e-5
poly2_data['price'] = set_2['price'] # add price to the data since it's the target
model2 = graphlab.linear_regression.create(poly2_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model2.coefficients.print_rows(num_rows=16)

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   -25115.9044254   |   1573017.12178   |
|   power_1   |  None |    783.49380028    |   9293.98471763   |
|   power_2   |  None |  -0.767759302942   |   23.3971636819   |
|   power_3   |  None | 0.000438766369254  |  0.0331210085962  |
|   power_4   |  None | -1.15169166858e-07 | 2.93724988829e-05 |
|   power_5   |  None | 6.84281360981e-12  | 1.72037616446e-08 |
|   power_6   |  None | 2.51195187082e-15  | 6.82484060777e-12 |
|   power_7   |  None | -2.06440608259e-19 | 1.84509003318e-15 |
|   power_8   |  None | -4.59673022352e-23 | 3.35295936015e-19 |
|   power_9   |  None | -2.71279236963e-29 | 3.78708790458e-23 |
|   power_10  |  None | 6.21818431996e-31  |        nan        |
|   power_11  |  None | 6.51741386006e-35  |        nan        |
|   power_12  |  None | -

In [54]:
poly3_data = polynomial_sframe(set_3['sqft_living'], 15)
my_features = poly3_data.column_names() # get the name of the features
l2_small_penalty = 1e-5
poly3_data['price'] = set_3['price'] # add price to the data since it's the target
model3 = graphlab.linear_regression.create(poly3_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model3.coefficients.print_rows(num_rows=16)

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   462426.565731    |        nan        |
|   power_1   |  None |   -759.251842854   |        nan        |
|   power_2   |  None |    1.0286700473    |        nan        |
|   power_3   |  None | -0.000528264527386 |        nan        |
|   power_4   |  None | 1.15422908385e-07  |        nan        |
|   power_5   |  None | -2.26095948062e-12 |        nan        |
|   power_6   |  None | -2.08214287571e-15 |        nan        |
|   power_7   |  None | 4.08770475709e-20  |        nan        |
|   power_8   |  None |  2.570791329e-23   |        nan        |
|   power_9   |  None | 1.24311265196e-27  |        nan        |
|   power_10  |  None | -1.72025834939e-31 |        nan        |
|   power_11  |  None | -2.96761071315e-35 |        nan        |
|   power_12  |  None | -

In [55]:
poly4_data = polynomial_sframe(set_4['sqft_living'], 15)
my_features = poly4_data.column_names() # get the name of the features
l2_small_penalty = 1e-5
poly4_data['price'] = set_4['price'] # add price to the data since it's the target
model4 = graphlab.linear_regression.create(poly4_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model4.coefficients.print_rows(num_rows=16)

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   -170240.032842   |   1543796.62885   |
|   power_1   |  None |   1247.59034541    |   9978.42791287   |
|   power_2   |  None |   -1.22460912177   |   26.7138477947   |
|   power_3   |  None | 0.000555254626344  |  0.0392072641565  |
|   power_4   |  None | -6.3826237386e-08  | 3.51264407588e-05 |
|   power_5   |  None | -2.20215991142e-11 | 2.00572757169e-08 |
|   power_6   |  None | 4.81834694285e-15  | 7.23449143456e-12 |
|   power_7   |  None | 4.21461612787e-19  | 1.43057715883e-15 |
|   power_8   |  None | -7.99880736276e-23 |        nan        |
|   power_9   |  None | -1.32365897487e-26 |        nan        |
|   power_10  |  None | 1.60197622166e-31  |  5.6453113885e-27 |
|   power_11  |  None | 2.39904337319e-34  | 9.66784867829e-31 |
|   power_12  |  None | 2

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [56]:
poly1_data = polynomial_sframe(set_1['sqft_living'], 15)
my_features = poly1_data.column_names() # get the name of the features
l2_small_penalty = 1e5
poly1_data['price'] = set_1['price'] # add price to the data since it's the target
model1 = graphlab.linear_regression.create(poly1_data, target = 'price', verbose=False,features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model1.coefficients.print_rows(num_rows=16)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   530317.024516   |   1890090.19151   |
|   power_1   |  None |   2.58738875673   |   9034.21455077   |
|   power_2   |  None |  0.00127414400592 |    18.000330944   |
|   power_3   |  None | 1.74934226932e-07 |   0.019765845458  |
|   power_4   |  None | 1.06022119097e-11 | 1.33158533722e-05 |
|   power_5   |  None | 5.42247604482e-16 | 5.77475960841e-09 |
|   power_6   |  None | 2.89563828343e-20 | 1.63140263499e-12 |
|   power_7   |  None | 1.65000666351e-24 | 2.91195278612e-16 |
|   power_8   |  None | 9.86081528409e-29 | 2.85000603886e-20 |
|   power_9   |  None | 6.06589348254e-33 |        nan        |
|   power_10  |  None |  3.7891786887e-37 | 5.26097974934e-29 |
|   power_11  |  None | 2.38223121312e-41 | 3.77182424405e-33 |
|   power_12  |  None | 1.49847969215e-4

In [57]:
poly2_data = polynomial_sframe(set_2['sqft_living'], 15)
my_features = poly2_data.column_names() # get the name of the features
l2_small_penalty = 1e5
poly2_data['price'] = set_2['price'] # add price to the data since it's the target
model2 = graphlab.linear_regression.create(poly2_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model2.coefficients.print_rows(num_rows=16)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   519216.897383   |   2167962.96521   |
|   power_1   |  None |   2.04470474182   |   12809.1515268   |
|   power_2   |  None |  0.0011314362684  |   32.2464286313   |
|   power_3   |  None | 2.93074277549e-07 |  0.0456480218892  |
|   power_4   |  None | 4.43540598453e-11 |  4.0481752482e-05 |
|   power_5   |  None | 4.80849112204e-15 | 2.37105608016e-08 |
|   power_6   |  None | 4.53091707826e-19 | 9.40612881847e-12 |
|   power_7   |  None | 4.16042910575e-23 | 2.54293917341e-15 |
|   power_8   |  None | 3.90094635128e-27 | 4.62111417353e-19 |
|   power_9   |  None |  3.7773187602e-31 | 5.21943862496e-23 |
|   power_10  |  None | 3.76650326842e-35 |        nan        |
|   power_11  |  None | 3.84228094754e-39 |        nan        |
|   power_12  |  None | 3.98520828414e-4

In [58]:
poly3_data = polynomial_sframe(set_3['sqft_living'], 15)
my_features = poly3_data.column_names() # get the name of the features
l2_small_penalty = 1e5
poly3_data['price'] = set_3['price'] # add price to the data since it's the target
model3 = graphlab.linear_regression.create(poly3_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model3.coefficients.print_rows(num_rows=16)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   522911.518048   |        nan        |
|   power_1   |  None |   2.26890421877   |        nan        |
|   power_2   |  None |  0.00125905041842 |        nan        |
|   power_3   |  None | 2.77552918155e-07 |        nan        |
|   power_4   |  None |  3.2093309779e-11 |        nan        |
|   power_5   |  None | 2.87573572364e-15 |        nan        |
|   power_6   |  None | 2.50076112671e-19 |        nan        |
|   power_7   |  None | 2.24685265906e-23 |        nan        |
|   power_8   |  None | 2.09349983135e-27 |        nan        |
|   power_9   |  None | 2.00435383296e-31 |        nan        |
|   power_10  |  None | 1.95410800249e-35 |        nan        |
|   power_11  |  None | 1.92734119456e-39 |        nan        |
|   power_12  |  None | 1.91483699013e-4

In [59]:
poly4_data = polynomial_sframe(set_4['sqft_living'], 15)
my_features = poly4_data.column_names() # get the name of the features
l2_small_penalty = 1e5
poly4_data['price'] = set_4['price'] # add price to the data since it's the target
model4 = graphlab.linear_regression.create(poly4_data, target = 'price',verbose=False, features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
model4.coefficients.print_rows(num_rows=16)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   513667.087087   |   2041482.90233   |
|   power_1   |  None |   1.91040938244   |   13195.2548642   |
|   power_2   |  None |  0.00110058029175 |   35.3258081465   |
|   power_3   |  None | 3.12753987879e-07 |   0.051846828738  |
|   power_4   |  None | 5.50067886825e-11 | 4.64504371162e-05 |
|   power_5   |  None | 7.20467557825e-15 | 2.65233027965e-08 |
|   power_6   |  None | 8.24977249384e-19 | 9.56673326956e-12 |
|   power_7   |  None | 9.06503223498e-23 | 1.89176395105e-15 |
|   power_8   |  None | 9.95683160453e-27 |        nan        |
|   power_9   |  None | 1.10838127982e-30 |        nan        |
|   power_10  |  None | 1.25315224143e-34 | 7.46523632876e-27 |
|   power_11  |  None | 1.43600781402e-38 | 1.27845516761e-30 |
|   power_12  |  None |  1.662699678e-42

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [60]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [61]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation
print (n)
for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

19396
0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [62]:
validation4 = train_valid_shuffled[5818:7758]

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [63]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [64]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 |  665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 |  645000.0 |   3.0    |    2.5    |
| 4139480190 | 2014-09-16 00:00:00+00:00 | 1153000.0 |   3.0    |    3.25   |
| 7237300290 | 2015-03-26 00:00:00+00:00 |  338000.0 |   5.0    |    2.5    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   59

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [65]:
train4 = train_valid_shuffled[0:5818].append(train_valid_shuffled[7758:19397])


To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [66]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [68]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    n = len(data)
    validation_error =[]
    for i in range(0,k-1):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation_set = data[start:end+1]
        training_set = data[0:start].append(data[end+1:len(data)+1])
        model = graphlab.linear_regression.create(training_set, target = output_name,verbose=False, features = features_list,l2_penalty=l2_penalty,validation_set=None)
        residuals_list = model.predict(validation_set[features_list]) - validation_set[output_name]
    r_square_list = residuals_list*residuals_list
    return r_square_list.mean()    

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [69]:
poly_data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
my_features = poly_data.column_names() # get the name of the features
poly_data['price'] = train_valid_shuffled['price'] # add price to the data since it's the target
l2_penalty = np.logspace(1, 7, num=13)
#print(l2_penalty[0])
k=10
output ='price'
validation_error =0
validation_errroset=[]
for i in xrange(len(l2_penalty)):
    validation_error = k_fold_cross_validation(k,l2_penalty[i], poly_data, output, my_features)
    validation_errroset.append(validation_error)
print validation_errroset


[58938861275.75689, 59085562732.27576, 59264633904.32933, 59467134583.644394, 59794997505.93269, 60921084399.29846, 65742371149.20213, 79155941831.54645, 96917195489.48543, 108835483678.41148, 114228951231.37352, 116233961682.53076, 116909333519.71263]


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [45]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.
poly_full_data = polynomial_sframe(train_valid['sqft_living'], 15)
poly_full_test_data = polynomial_sframe(test['sqft_living'], 15)
my_features = poly_full_data.column_names() # get the name of the features
my_features_test = poly_full_test_data.column_names()
l2_small_penalty = 10
poly_full_data['price'] = train_valid['price']
poly_full_test_data['price'] = test['price']
# add price to the data since it's the target
model_full = graphlab.linear_regression.create(poly_full_data, target = 'price', features = my_features,l2_penalty=l2_small_penalty,validation_set=None)
residuals_list = model.predict(polynomial_sframe(test['sqft_living'], 15)) - test['price']
r_square_list = residuals_list*residuals_list
print r_square_list.sum()


Linear regression:

--------------------------------------------------------

Number of examples          : 19396

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.030748     | 2320998.995562     | 247476.154615 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

1.23283298123e+14


Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

In [ ]:
model = graphlab.linear_regression.create(train_valid, target = train_valid['price'],verbose=False, features = features_list,l2_penalty=l2_penalty,validation_set=None)
        residuals_list = model.predict(validation_set[my_features]) - validation_set[output_name]
    r_square_list = residuals_list*residuals_list
    return r_square_list.mean()    

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [75]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted_price = model.predict(data)
    # Then compute the residuals/errors
    residuals = predicted_price - outcome
#     print residuals
    # Then square and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)

In [72]:
def k_fold_cross_validations(k, l2_penalty, data, output_name, features_list):
    rss = []
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation_data = data[start:end+1]
        train_data = data[0:start].append(data[end+1:])
        model = graphlab.linear_regression.create(train_data, target = output_name, verbose=None, features = features_list, l2_penalty=l2_penalty, validation_set = None)
        rss.append(get_residual_sum_of_squares(model, validation_data, validation_data[output_name]))
    return np.mean(rss)

In [73]:
l2_penalty_new = np.logspace(1, 7, num=13)

polyNorminal15Data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
featurelist = polyNorminal15Data.column_names()
polyNorminal15Data['price'] = train_valid_shuffled['price']

In [74]:
rssList = []
for penalty in l2_penalty_new:
    rssList.append(k_fold_cross_validations(10, penalty, polyNorminal15Data, 'price', featurelist))
print rssList

[491826427768997.69, 287504229919123.38, 160908965822178.22, 122090967326083.59, 121192264451214.88, 123950009289897.62, 136837175247519.03, 171728094842297.41, 229361431260422.69, 252940568728599.84, 258682548441132.34, 262819399742234.16, 264889015377543.81]


In [76]:
min(rssList)

121192264451214.88

In [77]:
l2_penalty_new[4]

1000.0

In [79]:
polynomialDataFullFinal = polynomial_sframe(train_valid['sqft_living'], 15)
featurelist_final = polynomialDataFullFinal.column_names()
polynomialDataFullFinal['price'] = train_valid['price']

In [81]:
final_model = graphlab.linear_regression.create(polynomialDataFullFinal,target='price',features=featurelist_final,verbose=False,l2_penalty=1000,validation_set=None)

In [82]:
test_poly_data = polynomial_sframe(test['sqft_living'], 15)
test_poly_data['price'] = test['price']

In [83]:
print get_residual_sum_of_squares(final_model, test_poly_data, test_poly_data['price'])

1.28780855058e+14
